In [1]:
import os, sys
sys.path.append("/home/files/feature_orthogonality")
from utils.session_config import setup_gpus

In [2]:
import tensorflow as tf
import cv2
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"
setup_gpus(memory_fraction=0.2)

1 Physical GPUs, 1 Logical GPUs


In [4]:
from tensorflow.keras.layers import InputLayer, Conv2D, MaxPool2D, Flatten, Dense, ReLU, BatchNormalization, GlobalAveragePooling2D
from utils.BEP import Back_etching_propagation

In [5]:
(train_x, train_y), (test_x, test_y) = tf.keras.datasets.mnist.load_data()
train_x = train_x[:,:,:,tf.newaxis]/255.
test_x = test_x[:,:,:,tf.newaxis]/255.

print(train_x.shape)
print(train_y.shape)

x = tf.data.Dataset.from_tensor_slices(train_x)
y = tf.data.Dataset.from_tensor_slices(train_y)
ds = tf.data.Dataset.zip((x, y))
ds = ds.shuffle(len(train_x), reshuffle_each_iteration=False).batch(32)

(60000, 28, 28, 1)
(60000,)


In [6]:
trained_model = tf.keras.models.load_model("/home/files/vision_tree/train/ckpt/mnist_cnn/10classes.h5")
# bep_algorithm = Back_etching_propagation(trained_model)
# #etching start
# bep_algorithm.categorical_etching(ds, num_class=10, save_path='10classes')

In [9]:
bep_algorithm = Back_etching_propagation(trained_model)

2 layer name :  conv2d_3
Conv2D
0 layer name :  re_lu_3
0 layer name :  max_pooling2d_2
2 layer name :  conv2d_4
Conv2D
0 layer name :  re_lu_4
0 layer name :  max_pooling2d_3
2 layer name :  conv2d_5
Conv2D
0 layer name :  re_lu_5
0 layer name :  global_average_pooling2d_1
2 layer name :  dense_1
Dense


In [10]:
bep_algorithm.model.summary()

Model: "functional_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
tf_op_layer_strided_slice (T [(None, 28, 28, 1, 1)]    0         
_________________________________________________________________
etching__layer (Etching_Laye (None, 28, 28, 1, 16)     16        
_________________________________________________________________
functional_1 (Functional)    [(None, 26, 26, 16), (Non 160       
_________________________________________________________________
re_lu_3 (ReLU)               (None, 26, 26, 16)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 16)        0         
_________________________________________________________________
tf_op_layer_strided_slice_1  [(None, 13, 13, 16, 1)]  

In [11]:
model = bep_algorithm.model

In [14]:
bep_algorithm.etching_layer_indice

[2, 7, 12, 17]

In [15]:
for layer in model.layers:
    layer.trainable = False   

In [17]:
model.layers[17].trainable = True

In [21]:
idx = 17
y = tf.keras.Input(shape=(), dtype=tf.int32)
i = model.layers[idx].output
_,o = model.layers[idx+1](i)

o = o*tf.one_hot(y, depth=10)
etching_model = tf.keras.Model(inputs=[model.input,y], outputs=o)

In [22]:
optimizer = tf.keras.optimizers.SGD(learning_rate=1E-3)

In [30]:
for x,y in ds.take(1):
    with tf.GradientTape() as tape:
        pred = etching_model([x,y], training=True)
        loss = tf.reduce_mean(pred)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

In [29]:
for x,y in ds.take(1):
    print(y)

tf.Tensor([5 5 9 9 3 4 3 5 7 1 5 7 5 2 6 9 3 2 0 8 1 7 4 3 7 8 7 0 4 7 1 6], shape=(32,), dtype=uint8)


In [31]:
model.layers[17].get_weights()

[array([[0.99996716, 1.0000153 , 0.99996245, 0.99992543, 1.0000244 ,
         1.0000287 , 0.99998444, 1.0000052 , 1.00002   , 1.0000066 ],
        [1.0000201 , 1.0000123 , 0.9999563 , 0.99999434, 1.0000019 ,
         1.0000665 , 1.0000135 , 0.9999681 , 0.999958  , 1.0000288 ],
        [1.0000043 , 0.9999956 , 1.0000224 , 0.99998426, 0.9999961 ,
         0.99994063, 1.0000055 , 0.9999123 , 1.0000153 , 0.99995345],
        [1.0000179 , 0.99996495, 1.0000134 , 0.9999798 , 0.99999946,
         1.0000094 , 1.0000073 , 1.0000044 , 1.0000038 , 0.9999763 ],
        [1.        , 1.        , 1.        , 1.        , 1.        ,
         1.        , 1.        , 1.        , 1.        , 1.        ],
        [1.0000042 , 0.999996  , 0.9999956 , 1.0000057 , 0.9999308 ,
         0.99998283, 0.99997014, 0.99998724, 1.0000068 , 0.9999936 ],
        [0.99997234, 0.9999686 , 0.9999647 , 0.999991  , 0.999973  ,
         1.0000476 , 1.0000029 , 0.9998433 , 1.0000054 , 0.99996996],
        [0.9999915 , 1.0000

In [54]:
bep_algorithm = Back_etching_propagation(trained_model)
model = bep_algorithm.model

2 layer name :  conv2d_3
Conv2D
0 layer name :  re_lu_3
0 layer name :  max_pooling2d_2
2 layer name :  conv2d_4
Conv2D
0 layer name :  re_lu_4
0 layer name :  max_pooling2d_3
2 layer name :  conv2d_5
Conv2D
0 layer name :  re_lu_5
0 layer name :  global_average_pooling2d_1
2 layer name :  dense_1
Dense


In [55]:
optimizer = tf.keras.optimizers.SGD(learning_rate=1E-3)

In [35]:
idx = 17
y = tf.keras.Input(shape=(), dtype=tf.int32)
i = model.layers[idx].output
_,o = model.layers[idx+1](i)

o = o*tf.one_hot(y, depth=10)
etching_model = tf.keras.Model(inputs=[model.input,y], outputs=o)

In [39]:
etching_model.layers[-5].trainable

True

In [51]:
etching_model.layers

In [53]:
etching_model.layers[-10].name

'etching__layer_6'

In [105]:
for x,y in ds.take(1):
    with tf.GradientTape(persistent=True) as tape:
        pred = trained_model(x, training=True)
        loss = tf.reduce_mean(pred)

In [78]:
model.layers[-2].name

'etching__layer_11'

In [79]:
model.layers[-7].name

'etching__layer_10'

In [109]:
k = tape.jacobian(pred, trained_model.layers[-4].weights)

In [110]:
k

[<tf.Tensor: shape=(32, 10, 3, 3, 16, 16), dtype=float32, numpy=
 array([[[[[[ 1.07320841e-06, -5.65648520e-07, -7.11445750e-07, ...,
              3.77199996e-07, -1.23231914e-07,  1.48481671e-08],
            [ 3.55127759e-06, -1.87174714e-06, -2.35026323e-06, ...,
              1.64047697e-06, -7.98784242e-07,  1.15162243e-06],
            [ 3.00240663e-06, -1.58245757e-06, -2.18269815e-06, ...,
              1.43571356e-06, -6.34984417e-07,  1.27295505e-06],
            ...,
            [ 5.05676144e-06, -2.66523239e-06, -2.82405927e-06, ...,
              2.82629208e-06, -1.28027091e-06,  2.23453594e-06],
            [ 1.41204760e-06, -7.44238037e-07, -8.67801987e-07, ...,
              5.45182274e-07, -1.83113130e-07,  1.40020205e-07],
            [ 1.57242027e-06, -8.28764655e-07, -1.05792651e-06, ...,
              8.46926071e-07, -4.40947076e-07,  3.36415098e-07]],
 
           [[ 8.10872613e-09, -4.27380975e-09, -6.42096509e-09, ...,
              7.35175698e-09, -5.46437962e

https://www.tensorflow.org/guide/advanced_autodiff?hl=ko

In [ ]:
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))